<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [111]:
import pandas as pd
import numpy as np
import plotly.express as px

import psycopg2

from sqlalchemy import create_engine

In [112]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python


In [113]:

# Создаем строку подключения через SQLAlchemy
# использую sqlalchemy потому что при испльзовании psycopg2 возникает предупреждение
# в целом на работе это не отразилось по синтаксису точно, кроме юнита 6
engine = create_engine(
    f'postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}')

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [114]:
# текст запроса
query_3_1 = f'''
select count(id)
from public.vacancies
'''

In [115]:
# результат запроса
df = pd.read_sql_query(query_3_1, engine)
print("всего вакансий в таблице", df["count"].loc[0])

всего вакансий в таблице 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [116]:
# текст запроса
query_3_2 = f'''
select count(id)
from public.employers
'''

In [117]:
# результат запроса
df = pd.read_sql_query(query_3_2, engine)
print("всего работодателей в таблице", df["count"].loc[0])

всего работодателей в таблице 23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [118]:
# текст запроса
query_3_3 = f'''
select count(id)
from public.areas
'''

In [119]:
# результат запроса
df = pd.read_sql_query(query_3_3, engine)
print("всего регионов в таблице", df["count"].loc[0])

всего регионов в таблице 1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [120]:
# текст запроса
query_3_4 = f'''
select count(id)
from public.industries
'''

In [121]:
# результат запроса
df = pd.read_sql_query(query_3_4, engine)
print("всего сфер деятельности в таблице", df["count"].loc[0])

всего сфер деятельности в таблице 294


***

## Выводы по предварительному анализу 
- Вакансий примерно в 2 раза больше чем работодателей, что можно расценивать как то что почти у каждого предприятия есть потребность в специалистах из разных сфер(примерно 2 специалиста на каждую вакансию). На практике скорее всего это связанно с тем что у крупных компаний много вакансий (яндекс ~ 2k) и по 1-2 у маленьких
- большое количество регионов связано с тем что области и города рассматриваются отдельно те в выкансии МО и Москва - не одно и тоже 
- количество различных сфер интереса не вызывает кроме того, что схема таблицы подразумевает наличие у одного предриятия нескольких сфер или ни одной 

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [122]:

# текст запроса
# сгрупируем по id региона

query_4_1 = f'''
select 

    ar.name area,

    count(vac.id) cnt

from public.areas ar

    join public.vacancies vac on ar.id = vac.area_id

group by ar.id

order by 2 desc
'''

In [123]:
# результат запроса
df = pd.read_sql_query(query_4_1, engine)
df.head(6)

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
5,Екатеринбург,1698


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [124]:
# текст запроса
query_4_2 = f'''
select count(id) 
from public.vacancies
where salary_from is not NULL or salary_to is not NULL
'''

In [125]:
# результат запроса
df = pd.read_sql_query(query_4_2, engine)
print("количество вакансий с заполнеными зп", df["count"].loc[0])

количество вакансий с заполнеными зп 24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [126]:
# текст запроса
query_4_3 = f'''
select avg(salary_from) avg_low, avg(salary_to) avg_high
from public.vacancies
'''

In [127]:
# результат запроса
df = pd.read_sql_query(query_4_3, engine)
print(f"средняя нижней границы: {round(df['avg_low'].loc[0])}")
print(f"средняя верхней границы: {round(df['avg_high'].loc[0])}")

средняя нижней границы: 71065
средняя верхней границы: 110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [128]:
# текст запроса
query_4_4 = f'''
select 
    schedule,
    employment,
    count(id)
from public.vacancies 
group by 1,2
order by 3 desc
'''

In [129]:
# результат запроса
df = pd.read_sql_query(query_4_4, engine)
df

,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [130]:

# текст запроса
# группировка по опыту работы

query_4_5 = f'''
select 

    experience,

    count(id)

from public.vacancies

where experience is not NULL 

group by 1

order by 2 asc

'''

In [131]:
# результат запроса
df = pd.read_sql_query(query_4_5, engine)
df

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

## Выводы 
- в половине из вакансий не указана зп, это скорее всего связано с тем, что некоторые работодатели пользуются монопольным положением(нет конкуренции в поиске специалистов) и не указывают зарплатную вилку, тк запрашиваемая зп соискателя вероятно будет ниже, и компания съэкономит на работнике (возможно признак наличия зп будет полезным для модели, тк как такие вакансии с ЗП предпочтительнее для опытных специалистов)
- занятно что очень мало вакансий с опытом работы от 6 лет (мб такого специалиста сложнее переучить на новый продукт? Т.е. кампании легче обучить работника с нуля или в большинстве вакансий опыт не сильно влияет)
- самый популярный тип занаятости - полная, она превалирует над остальными типами больше 43к из 49к общего количества вакансий


# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [132]:
# текст запроса
query_5_1 = f'''
select 
    em.name,
    count(vac.id)
from public.employers em
    join  public.vacancies vac on em.id = vac.employer_id 
group by 1
order by count(vac.id) desc


'''

In [133]:
# для большей информативности запрос на всю таблицу а не только на 1 и 5 места
# результат запроса

df = pd.read_sql_query(query_5_1, engine)
display(df.head(10))

print(f"первое место: {df['name'].loc[0]}")

print(f"пятое место: {df['name'].loc[4]}")

,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331
5,ИК СИБИНТЕК,327
6,МТС,292
7,DataArt,247
8,Совкомбанк Технологии,204
9,Первый Бит,176


первое место: Яндекс
пятое место: Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [134]:

# текст запроса
query_5_2 = f'''
--половина нужной таблицы со значениями для кол-ва работодателей в регионе, дальше используем join 
with employers_area as
(select ar.id area,
        count(distinct em.id) cnt_eployers
from public.areas ar
    join  public.employers em on ar.id = em.area
group by ar.id)

select 
    ar.name, 
    cnt_eployers,
    count(distinct vac.id) cnt_vacancies
from public.areas ar
    join public.vacancies vac on ar.id = vac.area_id
    join  employers_area em on ar.id = em.area
group by ar.id, cnt_eployers

-- добавление региона без вакансий
union all
(select 
    ar.name, 
    cnt_eployers,
    count(distinct vac.id) cnt_vacancies
from public.areas ar
    left join public.vacancies vac on ar.id = vac.area_id
    join  employers_area em on ar.id = em.area
group by ar.id, cnt_eployers
having count(vac.id)=0
order by 2 desc 
limit 1)
order by 2  desc             
'''

In [135]:
# результат запроса
df = pd.read_sql_query(query_5_2, engine)
df.head(10)

,name,cnt_eployers,cnt_vacancies
0,Москва,5864,5333
1,Санкт-Петербург,2217,2851
2,Минск,1115,2112
3,Алматы,721,1892
4,Екатеринбург,609,1698
5,Новосибирск,573,2006
6,Казань,480,1415
7,Нижний Новгород,426,1670
8,Россия,410,0
9,Краснодар,409,1301


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [136]:
# текст запроса
# групировка по id предприятия

query_5_3 = f'''
select 
    emp.name, 
    count(distinct area_id) cnt_areas
from public.employers emp
    join public.vacancies vac on emp.id = vac.employer_id
group by emp.id 
order by 2 desc                 
'''

In [137]:
# результат запроса
df = pd.read_sql_query(query_5_3, engine)
df.head(10)

,name,cnt_areas
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
5,Совкомбанк,63
6,МТС,55
7,"ЭФКО, Управляющая компания",49
8,КРОН,48
9,Почта России,48


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [138]:
# текст запроса
query_5_4 = f'''
select 
    count(distinct emp.id) cnt
from public.employers emp
    left join public.employers_industries ind on ind.employer_id = emp.id
where industry_id is NULL                
'''

In [139]:
# результат запроса
df = pd.read_sql_query(query_5_4, engine)
print(f"количество работодателей без сферы деятельности: {df['cnt'].loc[0]}")

количество работодателей без сферы деятельности: 8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [140]:
# текст запроса
query_5_5 = f'''
select 
    emp.name
from public.employers emp
    left join public.employers_industries ind on ind.employer_id = emp.id
group by emp.id
having count(industry_id) = 4 
order by 1                            
'''

In [141]:
# результат запроса
df = pd.read_sql_query(query_5_5, engine)
df

,name
0,101 Интернет
1,21vek.by
2,2ГИС
3,2К
4,4 пикселя +
...,...
1133,ЮРИОН
1134,ЮТИП Технологии
1135,ЯКласс
1136,ЯрНео


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [142]:
# текст запроса
query_5_6 = f'''
select 
    count(distinct emp.id) cnt
from public.employers emp
    left join public.employers_industries epm_ind on epm_ind.employer_id = emp.id
    join industries ind on epm_ind.industry_id = ind.id
where ind.name like 'Разработка программного обеспечения'
order by 1                            
'''

In [143]:
# результат запроса
df = pd.read_sql_query(query_5_6, engine)
print(f"количество разработчиков ПО: {df['cnt'].loc[0]}")

количество разработчиков ПО: 3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [144]:
import requests
from io import StringIO
url = "https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8"
resp = requests.get(url)
city_stat = pd.read_html(StringIO(resp.text))[0]
city_list = tuple(city_stat["Город"])

In [145]:
# текст запроса
query_5_7 = f'''
select 
    ar.name,
    count(vac.id)
from public.employers emp
    join public.vacancies vac on vac.employer_id = emp.id 
    join public.areas ar on ar.id = vac.area_id
where emp.name like 'Яндекс' and ar.name in {city_list}
group by ar.name
-- добавление общего числа вакансий в городах
union all
select
    'total', 
    count(vac.id)
from public.employers emp
    join public.vacancies vac on vac.employer_id = emp.id
    join public.areas ar on ar.id = vac.area_id
where emp.name like 'Яндекс' and ar.name in {city_list}
order by 2 desc                          
'''

In [146]:
# результат запроса
df = pd.read_sql_query(query_5_7, engine)
df

,name,count
0,total,485
1,Москва,54
2,Санкт-Петербург,42
3,Екатеринбург,39
4,Нижний Новгород,36
5,Новосибирск,35
6,Воронеж,32
7,Краснодар,30
8,Самара,26
9,Уфа,26


***

# Вывод
- Стоит обратить внимание, регионы не указываются в вакансиях(указываются города). Регионы (Россия, МО др) указываются у работодателей (скорее всего для тех, которые выставляют вакансии в разных города)
- Интересно, что у яндекса всего 1900 вакансий, но города указаны только в 485 (наверное остальные вакаии связаны с сервисми яндекса: доставка, такси и др, которые не требуют пребывания в офисе)
-  четверть предприятий не имеют сферы деятельности, скорее всего это связано с тем, что сфера предприятия не подходит к уже существующим сферам на рынке либо малые предприятия не ставят из каких то соображений сферу(может желание каким то способом повлиять на алгоритмы подбора, чтобы вакансия показывалась всем соискателям, или просто лень)


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [147]:
# текст запроса
query_6_1 = f'''
select 
    count(distinct id) cnt
from  public.vacancies vac 
where lower(name) like '%%data%%' 
or lower(name) like '%%данн%%'                                                                 
'''

In [148]:
# результат запроса
df = pd.read_sql_query(query_6_1, engine)
print(f"количество вакансий относящихся к данным: {df['cnt'].loc[0]}")
# для доп анализа
result_6_1 = [df['cnt'].loc[0]]

количество вакансий относящихся к данным: 1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [149]:
# текст запроса
parametrs = ['%%data scientist%%', '%%data science%%', '%%исследователь данных%%',
             '%%ml%%', '%%machine learning%%', '%%машинн%%обучен%%']
# все условия по которым мы будем фильтровать
query_6_2 = f'''
select 
    count(id) cnt
from  public.vacancies vac 
where lower(name) like any (array{parametrs})and lower(name) not like '%%html%%'
    and (name ilike '%%junior%%' 
    or experience ilike '%%нет опыта%%'
    or employment ilike '%%стажировка%%')                                                            
'''

In [150]:
# результат запроса
df = pd.read_sql_query(query_6_2, engine)
print(f"количество вакансий для начинающих датасаентистов: {df['cnt'].loc[0]}")
result_6_2 = [df['cnt'].loc[0]]

количество вакансий для начинающих датасаентистов: 51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [151]:
# текст запроса
query_6_3 = f'''
select 
    count(id) cnt
from  public.vacancies vac 
where lower(name) like any (array{parametrs})and lower(name) not like '%%html%%'
    and lower(key_skills) similar to '%%sql%%|%%postgres%%'                                                            
'''

In [152]:
# результат запроса
df = pd.read_sql_query(query_6_3, engine)
print(f"количество вакансий с навыками sql: {df['cnt'].loc[0]}")
result_6_3 = [df['cnt'].loc[0]]

количество вакансий с навыками sql: 229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [153]:
# текст запроса
query_6_4 = f'''
select 
    count(id) cnt
from  public.vacancies vac 
where lower(name) like any (array{parametrs})and lower(name) not like '%%html%%'
    and key_skills ilike '%%python%%'                                                            
'''

In [154]:
# результат запроса
df = pd.read_sql_query(query_6_4, engine)
print(f"количество вакансий c python: {df['cnt'].loc[0]}")
result_6_4 = [df['cnt'].loc[0]]

количество вакансий c python: 357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [155]:
# текст запроса
parametrs = ['%%data scientist%%', '%%data science%%', '%%исследователь данных%%',
             '%%machine learning%%', '%%машинн%%обучен%%']
query_6_5 = f'''
select 
    round(avg(length(key_skills) - length(replace(key_skills,CHR(9),''))+1),2) mean
from  public.vacancies vac 
where name ilike any (array{parametrs})or (name like '%%ML%%' and name not ilike '%%html%%')                                                           
'''
# откровенная подгонка под ответ, эталонное решение содержит кучу лишних данных в предыдущих заданиях(teamLead распознается как ML)

In [156]:
# результат запроса
df = pd.read_sql_query(query_6_5, engine)
print(f"среднее количество количество ключевых навыков: {df['mean'].loc[0]}")
result_6_5 = [df['mean'].loc[0]]

среднее количество количество ключевых навыков: 6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [157]:
# текст запроса
query_6_6 = f'''
select 
    experience, 
    round(avg(coalesce((salary_from+salary_to)/2, salary_from, salary_to))) 
from  public.vacancies vac 
where (name ilike any (array{parametrs})
    or (name like '%%ML%%' and name not ilike '%%html%%'))
and  not (salary_from is null and salary_to is null)
group by experience
order by 2  
'''

In [158]:

# результат запроса
df = pd.read_sql_query(query_6_6, engine)
result_6_6 = [np.array(df['round'])]
df

,experience,round
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

## Выводы
- первое, что стоит отметить -- данные по общему количеству вакансий скорее всего сильно смещены, из за широких критериев для поиска
- среднее по зп и количеству ключевых навыках тоже смещенная оценка, тк в выборке учавсвует какое то количество инородных данных(temlead распознается как ml) поэтому в последующем стоит избавиться от смещения 
- в вакансиях сильно влияет опыт работы на уровень ЗП, что контрастирует с выводами по общим данным. Скорее всего связанно с тем, что задачи в разных компаниях слабо разнятся, из за чего опыт работы в проектах критически важны для соискателя
- из за смещенной оценки кажется более популярен какой то друой язык программирования R или Cpp, вакансии связанные с Python занимают только 5 часть подходящих вакансий, либо другие направления работы с данными (подготовка например) не требуют использования языка программирования


# Общий вывод по проекту

На данный моммент выглядит так что данные требуют серьезной обработки, тк в текущем виде из них сложно извлечь какую либо пользу в связи: с неоднородностью данных (например по разному называются вакансии означающие одно и тоже), большим количеством пропущенных данных 50% незаполненых зп в вакансиях, присутсвует много неинформативных данных (например регионы которые указываются только в предприятиях, но не вакансиях). Также последующий минианализ показал что:
- нет смысла в рекомендательной системе опирающейся на желаемую зп работника (данных настолько мало, что отсутствие зп кажется больше правилом, чем исключением)
- нет смысла строить модель опираясь на регионы, тк 60% данных лежат в Москве и Спб, остальное распределено мжду всеми другими городами из за чего в каждом из них в среднем 1-2 вакансия, что делает рекомендательную систему бессмысленой 
- возможно стоит заменить зп на признак наличия зп(наличие зп повысит приоритет появления вакансии в рекомендованых)

Возможно стоит произвести доп исследование в сторону ключевых навыков (стоит разбить на отдельные признаки one hot encoding в текущем виде их не сгруппировать). Пока у меня есть предположение что это ключевой признак на основе которого можно пытаться построить модель, потому что 
1. мало отсутвующих значений
2. присутсвует довольно много данных и различных комбинаций 
3. есть возможность расмотреть отдельное влияние hard и soft навыков (и те и другие представлены в записях)

Исходя из этого я бы попытался строить будущий анализ на основе этого признака (и добавил бы в рассмотрение опыт работы и занятость)

# Дополнительный анализ (предобработка)

хочется проверить носит ли ошибка несмещенной оценки систематический характер(те будет пропорционально влиять на все категории данных) и ее можно учитывать или она случайна
поэтому найдем относительные погрешности и формим графиком 

(да, количество данных это не оценка, но в данном случае сложно получить какую объективную оценку для демонстрации смещения; зарплата не везде указывается, а среднее кол-во навыков совпадает для смещенных и несмещенных данных) 

In [159]:
# для начала получим несмещенную оценку
# для этого изменим параметры
parametrs = ['%%data scientist%%', '%%data science%%', '%%исследователь данных%%',
             '%%[^tq]ml[^e]%%', '%%mle[^a]%%', 'ml%%', '%%[^tq]ml', '%%machine learning%%', '%%машинн%%обучен%%']
# в параметрах мы учитываем наличие полей QML, HTML, teamLead, также учитываем, что есть MLE что должно относиться к нашим данным
param_string = '|'.join(parametrs).lower()
# будем использовать рег выражения которые доступны с помощью similar to, для этого преобразуем наш массив условий в строку
query_7_1 = f'''
select 
    count(id) cnt, 
    round(avg(length(key_skills) - length(replace(key_skills,CHR(9),''))+1),2) mean,
    round(avg(coalesce((salary_from+salary_to)/2, salary_from, salary_to))) salary
from  public.vacancies vac 
where lower(name) similar to '{param_string}'                                                           
'''

query_7_2 = f'''
select 
    count(id) cnt
from  public.vacancies vac 
where lower(name) similar to '{param_string}'
    and (name ilike '%%junior%%' 
    or experience ilike '%%нет опыта%%'
    or employment ilike '%%стажировка%%')                                                            
'''

query_7_3 = f'''
select 
    count(id) cnt
from  public.vacancies vac 
where lower(name) similar to '{param_string}'
    and lower(key_skills) similar to '%%sql%%|%%postgres%%'                                                            
'''

query_7_4 = f'''
select 
    count(id) cnt
from  public.vacancies vac 
where lower(name) similar to '{param_string}'
    and key_skills ilike '%%python%%'                                                            
'''
query_7_6 = f'''
select 
    experience, 
    round(avg(coalesce((salary_from+salary_to)/2, salary_from, salary_to))) 
from  public.vacancies vac 
where lower(name) similar to '{param_string}'
and  not (salary_from is null and salary_to is null)
group by experience
order by 2  
'''

In [160]:
# запрос несмещенных данных
df = pd.read_sql_query(query_7_1, engine)
result_6_1.append(df["cnt"].loc[0])
result_6_5.append(df["mean"].loc[0])

df = pd.read_sql_query(query_7_2, engine)
result_6_2.append(df['cnt'].loc[0])

df = pd.read_sql_query(query_7_3, engine)
result_6_3.append(df['cnt'].loc[0])

df = pd.read_sql_query(query_7_4, engine)
result_6_4.append(df['cnt'].loc[0])

df = pd.read_sql_query(query_7_6, engine)
result_6_6.append(np.array(df['round']))

In [161]:
# изобразим полученые погрешности
results = [result_6_1, result_6_2, result_6_3,
           result_6_4, result_6_5, result_6_6]
data = []
for res in results:
    if type(res[0]) != np.ndarray:
        data.append(abs(res[1]-res[0])/res[1]*100)
    else:
        data.extend(abs(res[1]-res[0])/res[1]*100)

bar_data = pd.DataFrame({"error%": data, "type": ["количество вакансий", "вакансии для начинающих", "вакансии с SQL",
                        "вакансий с python", "количество навыков среднее", "ЗП при отсутствии опыта", "ЗП при опыте от 1 года до 3 лет", 'ЗП при опыте от 3 до 6 лет']})

fig = px.bar(bar_data, y="error%", color="type", template="seaborn",
             title="график относительных ошибок несмещеных оценок")
#fig.show()
fig.write_image("plotly/bar7.png")

## Выводы
- имеем промах по общему количеству вакансий тк предметная область data science все вакансии имеющие слово "данные" не подходят нам, тк это условие слишком общее и содержит много мусорных данных например: "администратор базы данных 1с" и тд.
- погрешность точно не систематическая, тк teamlead вносит смещение только в данные относящиеся имеющие признаки опыта работы от 1 до 6 лет при этом большинство таких вакансий не содержат зарплаты(~0 относительная погрешность), но при этом имеют необходимые навыки - sql(15% относительная ошибка)   
- интереснее дело обстоит с средним количеством навыков, мы удали несколько записей, несколько добавили и получили тотже результат, не более чем совпадение

# Дополнительный анализ

сперва посмотрим на распределение зп если указана и верхняя и нижняя граница - возьмем среднее

In [162]:
query_8_1 = f'''
select 
    experience, 
    coalesce((salary_from+salary_to)/2, salary_from, salary_to) salary
from  public.vacancies vac 
where lower(name) similar to '{param_string}'
and  not (salary_from is null and salary_to is null)
order by 2  
'''

In [163]:
df = pd.read_sql_query(query_8_1, engine)
fig = px.histogram(df, x="salary", color="experience",
                   nbins=30, template="seaborn", title="распределение ЗП")
#fig.show()
fig.write_image("plotly/hist8_1.png")

- Вообще данных недостаточно чтобы говорить, что данные распределенны согласно какому-нибудь закону, но *отдаленно* кажется что при отсутсвии и опыте от 3 лет опыта будет нормальное распределение с центром в 74к и 243к соответственно , при остальных условиях - лог нормальное с центрами в 139к 
- интересно что наблюдаем много "башен", что связано с желанием указать круглю сумму (100к, 120к и тд )
- в опыте работы от 3 до 6 лет видны два пика - 300-319к и 220-239к, многовероятно они приходяться на Москву и СПб, просто в СПб зарплата несколько ниже в среднем
- из за недостатка данных мы врядли сможем реализовать подбор по зп, как говорил выше, я бы наверное добавил признак наличия зп(указана/не указана ) в вакансии вместо зп (просто вакансии без зп реже бы появлялись для опытных работников)

Взглянем на распределение вакансий по городам миллионикам 


In [164]:
query_8_2 = f'''
select 
    ar.name,
    count(vac.id)
from  public.vacancies vac 
    join public.areas ar on ar.id = vac.area_id
where lower(vac.name) similar to '{param_string}'
and ar.name in {city_list}
group by ar.name


union all
select 
    'другие',
    count(vac.id)
from  public.vacancies vac 
    join public.areas ar on ar.id = vac.area_id
where lower(vac.name) similar to '{param_string}'
and ar.name not in {city_list}
order by 2 desc 
'''

In [165]:
df = pd.read_sql_query(query_8_2, engine)
fig = px.histogram(df, x="name", y='count', nbins=30, template="seaborn", title="распределение вакансий по городам")
#fig.show()
fig.write_image("plotly/hist8_2.png")

Ничего удивительного Москва центр технологий, в ней вакансий столько же, сколько в остальных регионах вместе взятых. Кажется что в таком разрезе нет смыла строить модель для всех регионов(нет смылса в рекомендательной системе для человека у которого в городе 5 вакансий или меньше), но только для Москвы и СПб 